# Building a Small Language Model
This file will consist of the code necessary to build the components of a SLM/LLM, such as:
- Tokenizer
- Embeddings Layer
- Positional Embeddings

## Tokenizer

### Step 1: Creating Tokens

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters: ", len(raw_text))
print(len(raw_text[:99]))

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Embeddings Layer

## Transformer Architecture